In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "./../../")

In [3]:
from src import utils
from evaluation import evaluate
from training_utils import save_model

In [4]:
import os
import wandb
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random
from torch import optim
from sklearn import metrics
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

In [5]:
# Set random seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [6]:
scaler_dict = utils.open_object("./artifacts/feature_scaler.pkl")

In [7]:
config_dict = {}
config_dict['window_size'] = 24
config_dict['learning_rate'] = 0.0002
config_dict['batch_size'] = 12
config_dict['epochs'] = 200
config_dict['device'] = "cuda" if torch.cuda.is_available() else "cpu"
config_dict['input_size'] = 5
config_dict['hidden_size'] = 32
config_dict['output_size'] = config_dict['input_size']
config_dict['lstm_num_layers'] = 3
config_dict['dropout'] = 0.2

In [8]:
wandb_project = "QQQ Stock Price Prediction Based On LSTM"

In [9]:
display_name = f"window_size:{config_dict['window_size']} batch_size:{config_dict['batch_size']}"

In [10]:
wandb.init(
    project=wandb_project,
    job_type="train",
    config=config_dict,
    name=display_name
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nelsonlin0321. Use `wandb login --relogin` to force relogin


In [11]:
class Config:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)


config = Config(dictionary=config_dict)

### (1) Read Data

In [12]:
df = pd.read_parquet("./artifacts/processed_data.parquet")

In [13]:
df = df.sort_values(by='date').reset_index(drop=True)

In [14]:
df.head()

,date,open,high,low,close,volume
0,1999-11-01,0.481644,0.480709,0.481546,0.479795,0.028304
1,1999-11-02,0.481644,0.480709,0.481156,0.479990,0.071394
2,1999-11-03,0.484156,0.483017,0.485037,0.485023,0.129351
3,1999-11-04,0.489107,0.485494,0.487338,0.487883,0.139565
4,1999-11-05,0.493593,0.490750,0.492627,0.490523,0.096587


### (2) Split the data into train and test sets

In [15]:
features_cols = ["volume", "high", "low", "open", "close"]

df_data = df[features_cols].copy()

In [16]:
df_data.head()

,volume,high,low,open,close
0,0.028304,0.480709,0.481546,0.481644,0.479795
1,0.071394,0.480709,0.481156,0.481644,0.479990
2,0.129351,0.483017,0.485037,0.484156,0.485023
3,0.139565,0.485494,0.487338,0.489107,0.487883
4,0.096587,0.490750,0.492627,0.493593,0.490523


In [17]:
train_size = int(0.8 * len(df_data))
df_train = df_data[:train_size]
df_test = df_data[train_size:]

### (3) Create Sequence of input and output

In [18]:
# Custom dataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size

    def __len__(self):
        return len(self.data) - self.window_size

    def __getitem__(self, idx):
        inputs = torch.tensor(
            self.data[idx:idx+self.window_size], dtype=torch.float32)
        target = torch.tensor(
            self.data[idx+self.window_size], dtype=torch.float32)
        return inputs, target

In [19]:
train_dataset = TimeSeriesDataset(
    data=df_train.values, window_size=config.window_size)

In [20]:
test_dataset = TimeSeriesDataset(
    data=df_test.values, window_size=config.window_size)

In [21]:
train_loader = DataLoader(train_dataset, config.batch_size,
                          shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset,
                         config.batch_size, shuffle=False)

In [22]:
for inputs, targets in train_loader:
    break

In [23]:
inputs.shape  # batch_size, sequence_len, features

torch.Size([12, 24, 5])

In [24]:
targets.shape  # batch_size, features

torch.Size([12, 5])

### (4) Modeling

In [25]:
criterion = nn.MSELoss()

In [26]:
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, lstm_num_layers, hidden_size, output_size, device):
        super(BiLSTMModel, self).__init__()
        self.device = device
        self.lstm_num_layers = lstm_num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size,
                            num_layers=lstm_num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(2*self.lstm_num_layers, x.size(0), self.hidden_size).to(
            self.device)
        c0 = torch.zeros(2*self.lstm_num_layers, x.size(0), self.hidden_size).to(
            self.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)

        return out

In [27]:
model = BiLSTMModel(input_size=config.input_size, lstm_num_layers=config.lstm_num_layers,
                    hidden_size=config.hidden_size, output_size=config.output_size,
                    device=config.device)

In [28]:
x = torch.randn(config.batch_size, config.window_size, config.input_size)

In [29]:
with torch.no_grad():
    out = model(x)

In [30]:
criterion(out, targets)

tensor(0.0627)

In [31]:
criterion(out[:, -1], targets[:, -1])

tensor(0.0706)

In [32]:
# class self:
#     pass

In [33]:
# input_size = config.input_size
# lstm_num_layers = config.lstm_num_layers
# hidden_size = config.hidden_size
# output_size = config.output_size
# device = config.device

In [34]:
# self.device = device
# self.lstm_num_layers = lstm_num_layers
# self.hidden_size = hidden_size
# self.lstm = nn.LSTM(input_size, hidden_size,
#                     num_layers=lstm_num_layers, batch_first=True, bidirectional=True)
# self.fc = nn.Linear(hidden_size * 2, output_size)
# self.sigmoid = nn.Sigmoid()

In [35]:
# h0 = torch.zeros(2*self.lstm_num_layers, x.size(0), self.hidden_size).to(
#     self.device)
# c0 = torch.zeros(2*self.lstm_num_layers, x.size(0), self.hidden_size).to(
#     self.device)
# out, _ = self.lstm(x, (h0, c0))
# out = self.fc(out[:, -1, :])
# out = self.sigmoid(out)

In [36]:
# out.shape

In [37]:
# loss = criterion(out, targets)

In [38]:
# loss

In [39]:
# criterion(out[:, -1], targets[:, -1])

### (5) Training

In [40]:
scaler = scaler_dict['close']

In [41]:
evaluate(model=model, test_loader=test_loader,
         scaler=scaler, criterion=criterion)

Evaluating:: 100%|██████████| 100/100 [00:00<00:00, 155.05it/s]


{'eval_loss': 0.03720933845732361,
 'eval_price_loss': 0.042895716964267194,
 'eval_MEA': 161.55112}

In [42]:
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
best_eval_loss = float("inf")
model_version = "v1"
metrics_list = []
best_model_path = None

In [40]:
total_pbar = tqdm(total=len(train_loader)*config.epochs,
                  desc="Training", position=0, leave=True)

for epoch in range(config.epochs):

    train_loss_list = []
    price_loss_list = []
    prediction_list = []
    ground_truth_list = []

    model = model.train()
    for inputs, targets in train_loader:

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        price_loss = criterion(outputs[:, -1], targets[:, -1])

        loss.backward()
        optimizer.step()

        train_loss_list.append(loss.item())
        price_loss_list.append(price_loss.item())

        yhat = outputs[:, -1].detach().cpu().numpy()
        prediction_list.append(yhat)

        y = targets[:, -1].detach().cpu().numpy()
        ground_truth_list.append(y)

        total_pbar.update(1)

    improve = False
    model_metrics = evaluate(
        model, test_loader, scaler, criterion)
    eval_loss = model_metrics['eval_loss']

    if eval_loss <= best_eval_loss:
        improve = True
        best_checkpoint = epoch
        best_eval_loss = eval_loss

    train_loss = np.mean(train_loss_list)
    train_price_loss = np.mean(price_loss_list)

    predictions = np.concatenate(prediction_list)
    ground_truths = np.concatenate(ground_truth_list)

    predictions = np.exp(scaler.inverse_transform(
        predictions.reshape(-1, 1)))[:, 0]
    ground_truths = np.exp(scaler.inverse_transform(
        ground_truths.reshape(-1, 1)))[:, 0]

    mae = metrics.mean_absolute_error(ground_truths, predictions)
    model_metrics['best_eval_loss'] = best_eval_loss

    model_metrics['train_loss'] = train_loss
    model_metrics['train_price_loss'] = train_price_loss
    model_metrics['train_MAE'] = mae

    model_metrics["epoch"] = epoch
    model_metrics["best_epoch"] = best_checkpoint
    metrics_list.append(model_metrics)
    # wandb.log(model_metrics)

    if improve:
        save_dir = os.path.join("models", model_version)
        os.makedirs(save_dir, exist_ok=True)
        model_path = save_model(model, save_dir, epoch, model_metrics)
        best_model_path = model_path

    post_fix_message = {k: round(v, 4) for k, v in model_metrics.items()}
    total_pbar.set_postfix(post_fix_message)


total_pbar.close()
# wandb.finish()

Training:   6%|▌         | 4771/80400 [01:15<21:27, 58.73it/s, eval_loss=0.0062, eval_price_loss=0.0066, eval_MEA=74.4, best_eval_loss=0.0062, train_loss=0.0006, train_price_loss=0.0001, train_MAE=1.86, epoch=10, best_epoch=10]